In [14]:
import pandas as pd

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

# imported the CSV data
df = pd.read_csv("Filtered_Data.csv", sep = ",")
df.head()

,cvr_base_data_table.ts,cvr_base_data_table.impression_system_date,cvr_base_data_table.hour,cvr_base_data_table.conversion_status,cvr_base_data_table.cc_id_null_ind,cvr_base_data_table.ad_id,cvr_base_data_table.click_id,cvr_base_data_table.cc_id,cvr_base_data_table.hour_of_day,cvr_base_data_table.raw_publisher_id,...,cvr_base_data_table.click_conf_gmt_offset,cvr_base_data_table.click_state,cvr_base_data_table.click_city,cvr_base_data_table.ad_bid,cvr_base_data_table.ad_cpa_goal,cvr_base_data_table.ad_market_place_id,cvr_base_data_table.click_click_status,cvr_base_data_table.kmean_category_name,cvr_base_data_table.partner_id,cvr_base_data_table.version
0,20190106,2019-01-06,1,1,0,138154,646739319951006038,546739320882001606,20,8CUDMGV55,...,-500,ga,atlanta,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
1,20190106,2019-01-06,1,1,0,138154,646739319951006038,546739320882001606,20,8CUDMGV55,...,-500,ga,atlanta,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
2,20190106,2019-01-06,2,0,0,138154,646740715393002311,346740715723006848,20,8CUDMGV55,...,-600,al,birmingham,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
3,20190106,2019-01-06,4,0,0,138154,246748175931002645,446748176329007771,23,8CUDMGV55,...,-500,in,ft wayne,0.150000,NaN,1027,1,missing,8PRHGG6T9,1
4,20190106,2019-01-06,6,0,0,128699,246754980029002839,646754981260002933,22,8CUX8874N,...,-800,ca,mountain view,3.394224,NaN,1027,1,missing,8PRHGG6T9,1


In [15]:
# These columns contains redundant data, needs to be removed - only one type of value
print (df.columns)
print ('\n')
print ('Below are the list of columns names having only single values in entire columns')

for col in df.columns:
    if(df[col].nunique() == 1):
        print (col)

Index(['cvr_base_data_table.ts', 'cvr_base_data_table.impression_system_date',
       'cvr_base_data_table.hour', 'cvr_base_data_table.conversion_status',
       'cvr_base_data_table.cc_id_null_ind', 'cvr_base_data_table.ad_id',
       'cvr_base_data_table.click_id', 'cvr_base_data_table.cc_id',
       'cvr_base_data_table.hour_of_day',
       'cvr_base_data_table.raw_publisher_id',
       'cvr_base_data_table.raw_sub_publisher_id',
       'cvr_base_data_table.raw_sub_sub_publisher_id',
       'cvr_base_data_table.enriched_country',
       'cvr_base_data_table.enriched_derived_device',
       'cvr_base_data_table.enriched_derived_os',
       'cvr_base_data_table.enriched_publisher_domain',
       'cvr_base_data_table.ad_adgroup_id',
       'cvr_base_data_table.ad_advertiser_id',
       'cvr_base_data_table.ad_campaign_id', 'cvr_base_data_table.ad_keyword',
       'cvr_base_data_table.serving_key', 'cvr_base_data_table.ad_partner_id',
       'cvr_base_data_table.impression_state',
     

In [16]:
#Started removing above columns

In [17]:
df = df.drop('cvr_base_data_table.cc_id_null_ind', axis = 1)

In [18]:
df = df.drop('cvr_base_data_table.serving_key', axis = 1)
df = df.drop('cvr_base_data_table.ad_partner_id', axis = 1)
df = df.drop('cvr_base_data_table.ad_market_place_id', axis = 1)
df = df.drop('cvr_base_data_table.click_click_status', axis = 1)
df = df.drop('cvr_base_data_table.kmean_category_name', axis = 1)
df = df.drop('cvr_base_data_table.version', axis = 1)

In [19]:
df.head()

,cvr_base_data_table.ts,cvr_base_data_table.impression_system_date,cvr_base_data_table.hour,cvr_base_data_table.conversion_status,cvr_base_data_table.ad_id,cvr_base_data_table.click_id,cvr_base_data_table.cc_id,cvr_base_data_table.hour_of_day,cvr_base_data_table.raw_publisher_id,cvr_base_data_table.raw_sub_publisher_id,...,cvr_base_data_table.category_name,cvr_base_data_table.click_browser,cvr_base_data_table.click_user_datetime,cvr_base_data_table.raw_sync_partner_id,cvr_base_data_table.click_conf_gmt_offset,cvr_base_data_table.click_state,cvr_base_data_table.click_city,cvr_base_data_table.ad_bid,cvr_base_data_table.ad_cpa_goal,cvr_base_data_table.partner_id
0,20190106,2019-01-06,1,1,138154,646739319951006038,546739320882001606,20,8CUDMGV55,8PO1W66UO,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:48:40.883,1,-500,ga,atlanta,0.150000,NaN,8PRHGG6T9
1,20190106,2019-01-06,1,1,138154,646739319951006038,546739320882001606,20,8CUDMGV55,8PO1W66UO,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:48:40.883,1,-500,ga,atlanta,0.150000,NaN,8PRHGG6T9
2,20190106,2019-01-06,2,0,138154,646740715393002311,346740715723006848,20,8CUDMGV55,8PO1W66UO,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:11:55.723,1,-600,al,birmingham,0.150000,NaN,8PRHGG6T9
3,20190106,2019-01-06,4,0,138154,246748175931002645,446748176329007771,23,8CUDMGV55,8PO1W66UO,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 23:16:16.033,1,-500,in,ft wayne,0.150000,NaN,8PRHGG6T9
4,20190106,2019-01-06,6,0,128699,246754980029002839,646754981260002933,22,8CUX8874N,8PO4X3YG2,...,"finance,grants,,scholarships,,,financial,aid","google,chrome",2019-01-05 22:09:41.026,1,-800,ca,mountain view,3.394224,NaN,8PRHGG6T9


In [20]:
# Remove the timestamp data and impression system date data - results not dependent on the time frame
df = df.drop('cvr_base_data_table.ts', axis = 1)
df = df.drop('cvr_base_data_table.impression_system_date', axis = 1)

# Server hour do not matters
df = df.drop('cvr_base_data_table.hour', axis = 1)

# Data have only 3 values of *au* region, and conversion status 0 for all of them
df = df.drop('cvr_base_data_table.enriched_country', axis = 1)

In [21]:
df.head()

,cvr_base_data_table.conversion_status,cvr_base_data_table.ad_id,cvr_base_data_table.click_id,cvr_base_data_table.cc_id,cvr_base_data_table.hour_of_day,cvr_base_data_table.raw_publisher_id,cvr_base_data_table.raw_sub_publisher_id,cvr_base_data_table.raw_sub_sub_publisher_id,cvr_base_data_table.enriched_derived_device,cvr_base_data_table.enriched_derived_os,...,cvr_base_data_table.category_name,cvr_base_data_table.click_browser,cvr_base_data_table.click_user_datetime,cvr_base_data_table.raw_sync_partner_id,cvr_base_data_table.click_conf_gmt_offset,cvr_base_data_table.click_state,cvr_base_data_table.click_city,cvr_base_data_table.ad_bid,cvr_base_data_table.ad_cpa_goal,cvr_base_data_table.partner_id
0,1,138154,646739319951006038,546739320882001606,20,8CUDMGV55,8PO1W66UO,429821366,mobile,android,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:48:40.883,1,-500,ga,atlanta,0.150000,NaN,8PRHGG6T9
1,1,138154,646739319951006038,546739320882001606,20,8CUDMGV55,8PO1W66UO,429821366,mobile,android,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:48:40.883,1,-500,ga,atlanta,0.150000,NaN,8PRHGG6T9
2,0,138154,646740715393002311,346740715723006848,20,8CUDMGV55,8PO1W66UO,110622874,mobile,android,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 20:11:55.723,1,-600,al,birmingham,0.150000,NaN,8PRHGG6T9
3,0,138154,246748175931002645,446748176329007771,23,8CUDMGV55,8PO1W66UO,110622874,mobile,android,...,"shopping,consumer,resources,coupons,,,discount...","google,chrome",2019-01-05 23:16:16.033,1,-500,in,ft wayne,0.150000,NaN,8PRHGG6T9
4,0,128699,246754980029002839,646754981260002933,22,8CUX8874N,8PO4X3YG2,670653476,mobile,android,...,"finance,grants,,scholarships,,,financial,aid","google,chrome",2019-01-05 22:09:41.026,1,-800,ca,mountain view,3.394224,NaN,8PRHGG6T9


In [22]:
df['cvr_base_data_table.ad_cpa_goal'].value_counts()

200.00    23003
21.50     10316
23.00      9067
50.00      8564
10.00      5002
1.50       2891
150.00     1620
40.00      1500
175.00      378
1.75        301
35.00       233
17.50       126
31.00        79
33.00        21
Name: cvr_base_data_table.ad_cpa_goal, dtype: int64

In [23]:
df['cvr_base_data_table.click_browser'].value_counts()

google,chrome                61376
safari                       25944
edge                         16964
internet,explorer            11315
mozilla,firefox               3600
applewebkit                   3413
silk                          1356
opera,mini                     206
android,browser                 78
chromium                        23
unknown                         17
mozilla,like,unknown            10
nokia                           10
opera                            7
ie,mobile                        7
maxthon                          4
ucbrowser                        2
blackberry,browser               2
android,httpurlconnection        1
Name: cvr_base_data_table.click_browser, dtype: int64

In [47]:
df.info()
# print (df['cvr_base_data_table.conversion_status'].value_counts().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124335 entries, 0 to 124334
Data columns (total 31 columns):
cvr_base_data_table.conversion_status             124335 non-null int64
cvr_base_data_table.ad_id                         124335 non-null int64
cvr_base_data_table.click_id                      124335 non-null int64
cvr_base_data_table.cc_id                         124335 non-null int64
cvr_base_data_table.hour_of_day                   124335 non-null int64
cvr_base_data_table.raw_publisher_id              124335 non-null object
cvr_base_data_table.raw_sub_publisher_id          124335 non-null object
cvr_base_data_table.raw_sub_sub_publisher_id      124335 non-null object
cvr_base_data_table.enriched_derived_device       124335 non-null object
cvr_base_data_table.enriched_derived_os           124335 non-null object
cvr_base_data_table.enriched_publisher_domain     124335 non-null object
cvr_base_data_table.ad_adgroup_id                 124335 non-null int64
cvr_base_data_table

In [78]:
dd = {'cvr_base_data_table.conversion_status' : {
    'conversion' : 'sum',
    'click' : 'count'
}}
dfg = df.groupby('cvr_base_data_table.click_browser').agg(dd).reset_index()
dfg.columns = dfg.columns.droplevel(0)
tot = dfg['click'].sum()
dfg['cvr'] = dfg['conversion'] / dfg['click']
dfg['click % of total data'] = dfg['click'] / tot * 100
dfg = dfg.sort_values(by=['click % of total data'], ascending = False)
dfg.columns

Index(['', 'conversion', 'click', 'cvr', 'click % of total data'], dtype='object')

In [79]:
tots = df.shape[0]
for col in df.columns:
    if col == 'cvr_base_data_table.conversion_status':
        continue
    dfg = df.groupby(col).agg(dd).reset_index()
    dfg.columns = dfg.columns.droplevel(0)
    
    dfg['cvr'] = dfg['conversion'] / dfg['click']
    dfg['click percentage of total data'] = dfg['click'] / tots * 100
    dfg = dfg.sort_values(by=['click percentage of total data'], ascending = False)
    dfg.to_csv("Done/" + str(col) + ".csv")

In [80]:
df['cvr_base_data_table.cc_id']

0          546739320882001606
1          546739320882001606
2          346740715723006848
3          446748176329007771
4          646754981260002933
5          546756902206003613
6          446759863997003939
7          646780613334006319
8          346781811230007878
9          346785644736002846
10         346816301456007161
11         546734925973002292
12         346740583864006663
13         546741328661003974
14         546743857298007104
15         646745134564003589
16         446757492974003005
17         346777053601000254
18         546782303184005307
19         446786813063002174
20         146792650498009334
21         646797870392004150
22         346801229400002338
23         146803744514009936
24         446808760223009960
25         646735124703006804
26         646739517226004565
27         646754888986004642
28         146758597306000364
29         446768772178007339
                 ...         
124305    7147152775548002872
124306    7947153118883004899
124307    